# Blank Template for Developing

In [ ]:
%%javascript
window.IPython && (IPython.OutputArea.prototype._should_scroll = function(lines) { return false; })
// disable widget scrolling

In [ ]:
# Multi-Pass implementation of a trivial crossover system using the Quantiacs built-in backtester.

import xarray as xr           # xarray for data manipulation
import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library



def load_data(period):
    """This function loads the data. In the Documentation you will find more details
    on the different datasets you can load. You can inspect the source code in your
    root folder.
    """
    
    # loads cryptocurrency data for the Q17 crypto contest:
    return qndata.cryptodaily_load_data(tail = period, dims = ("time", "field", "asset"))
    # loads futures data for the Q15 futures contest:
    #return qndata.futures.load_data(tail = period, dims = ("time", "field", "asset"))



def strategy(data):
    """This function contains your strategy. It must returns allocation
    weights for all assets at a FIXED point in time (note isel(time=-1)).
    """
    
    close = data.sel(field="close")
    
    ma_slow = qnta.sma(close, 200).isel(time=-1)
    ma_fast = qnta.sma(close, 20).isel(time=-1)
    
    # go long on the assets when the fast moving average of the price
    # is larger than the slow one:
    weights = xr.where(ma_slow < ma_fast, 1, -1) # 1 - long position (buy), -1 - short position (sell)
    
    # this field tags cryptocurrencies which, at a given point in time,
    # are among the 10 ones with the largest market capitalization:
    is_liquid = data.sel(field="is_liquid")
    
    weights = weights * is_liquid
    weights = weights / 10.0

    return weights



# lookback_period is expressed in calendar days and it expresses the length
# of the rolling window used by the backtester. It must be large enough
# to include all indicators for your strategy (expressed in trading days).
# The smaller the window, the more efficient the evaluation.

weights = qnbt.backtest(
    competition_type = "crypto_daily_long_short", # "futures"
    load_data = load_data,
    lookback_period = 365*4,
    start_date = "2014-01-01",
    strategy=strategy
)

For improving speed you can implement your strategy using a **Single-Pass approach**. Your algorithm must return arrays for the allocation weights. In this implementation the full time series is accessible to your algo at any point in time, so you should make sure that no forward looking is taking place and your algo does not use future information for predicting the past. 

**IMPORTANT: Any** implementation in your Notebook (Single-Pass or Multi-Pass) will be processed **after submission on our servers** using a multi-pass approach in order to prevent forward looking. If the **Sharpe ratio** of your submission does not match your expectations from your Notebook research, please review your implementation: most likely some forward looking is taking place, for example by computing some global mean value which is then used for taking trading decisions.

```python
# Single-pass implementation (for prototyping)
import xarray as xr

import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.stats as qns

# load data:
data = qndata.cryptodaily_load_data(min_date="2013-05-01")

# calculate weights:
close = data.sel(field="close")
ma_slow = qnta.sma(close, 200)
ma_fast = qnta.sma(close, 20)
weights = xr.where(ma_fast > ma_slow, 1, -1)

# liquidity filter:
is_liquid = data.sel(field="is_liquid")

# set weights:
weights = weights * is_liquid
weights = weights / 10.0

# clean weights taking corner cases into account:
weights = qnout.clean(weights, data, "crypto_daily_long_short")

# check before submission:
qnout.check(weights, data, "crypto_daily_long_short")

# write results:
qnout.write(weights)

# calculate statistics for checking:
stats = qns.calc_stat(data, weights.sel(time=slice("2014-01-01",None)))
stats.to_pandas().tail()
```